In [1]:
from keras2plc import ST_writer
from keras2plc import keras_to_st_parser
from keras2plc.keras2plc import _get_io_dimensions
from keras2plc import keras2plc
import keras

folder = "test_ST_files/"
model_name = "Dense_v1"
model_file = "test_model.keras"


model = keras.saving.load_model(model_file)
i_,o_ = _get_io_dimensions(model)

reader = keras_to_st_parser(model,model_name, input_dim=i_, output_dim=o_)
layers_contents = reader.generate_struct_layers()
layersWeights_contents = reader.generate_struct_layer_weights()
bin_weights = reader.pack_weights_binary()


writer = ST_writer(model_name, layers_contents,layersWeights_contents)
writer.write_ST_files_to(folder, overwrite_if_exists=True)
writer.write_weights_file(bin_weights=bin_weights)

2024-04-19 17:41:31.306875: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 17:41:31.310564: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 17:41:31.370883: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-19 17:41:32.426114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
# minimal example

from keras2plc import keras2plc, get_example_usage
import keras 

folder = "test_ST_files/"
model_name = "Dense_v1"
model_file = "test_model.keras"

model = keras.saving.load_model(model_file)

keras2plc(model, model_name, folder, overwrite_if_model_exists=True)

print(get_example_usage(model, model_name))

The following code can be used to call the generated model:
        Assuming declared input/output for model:
        
            input : ARRAY[0..0] OF LREAL;
            result : ARRAY[0..0] OF LREAL;

        Then call as:

            FB_Dense_v1(pointer_input:=ADR(input), pointer_output:=ADS(result));      
        
        


In [2]:
print(layers_contents)
# print(layersWeights_contents)


num_layers : UINT := 4;
weights : Dense_v1_LayerWeights;
input : Layer := (num_neurons := 3,normalization := act_type.normalization);
layer_1 : Layer := (num_neurons := 31, activation := act_type.relu, pointer_weight:= ADR(weights.HiddenLayers1_weight),pointer_bias:= ADR(weights.HiddenLayers1_bias) );
layer_2 : Layer := (num_neurons := 24, activation := act_type.relu, pointer_weight:= ADR(weights.HiddenLayers2_weight),pointer_bias:= ADR(weights.HiddenLayers2_bias) );
output : Layer := (num_neurons := 1, activation := act_type.relu ,normalization := act_type.denormalization , pointer_weight:= ADR(weights.OutputLayer_weight),pointer_bias:= ADR(weights.OutputLayer_bias) );
layers : ARRAY[0..3] OF Layer :=[input, layer_1, layer_2, output];
layer_output : ARRAY[0..30] OF LREAL;
layer_input : ARRAY[0..30] OF LREAL;


In [3]:
writer.write_ST_files_to(folder, overwrite_if_exists=True)
reader.generate_weights_file(folder,True)
print(writer.create_example_usage(dims_input=2, dims_output=1))

The following code can be used to call the generated model:
        Assuming declared input/output for model:
        
            input : ARRAY[0..1] OF LREAL;
            result : ARRAY[0..0] OF LREAL;

        Then call as:

            FB_Dense_v1(pointer_input:=ADR(input), pointer_output:=ADS(result), nn:=GVL_Dense_v1.nn);      
        
        


In [5]:
import numpy as np
input = np.array([1,1,1]).reshape(1,3)
out = reader.model.predict(input)
print(out)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.50318885]]
